## For this process, we will primarily use PyGrib

---
### Before running through this notebook, make sure to complete these notebooks
- [2D Variables](2D-Vars.ipynb)
- [3D Variables](3D-Vars.ipynb)
- [Surface Variables](Surface_Variables.ipynb)
---
#### Running these notebooks will create graphics to visualize the climate change deltas, in addition to netCDF files that contain the data
---

#### Now that teverything is setup, let's import the neccessary libraries

In [1]:
import pygrib
import ncepgrib2

#### This is where you input where the NAM Analysis files are stored

In [2]:
# Directory all the NAM Analysis files are stored
nam_filepath = '/data/meso/a/jtrapp/formax/'

# Directory you would like the modified NAM files to be placed
new_filepath = '/data/keeling/a/mgrover4/b/CMIP5_to_WRF/modified_files/'

# Test file
file = 'namanl_218_20100510_1200_000.grb'

In [3]:
# Read in the data using pygrib
grbs = pygrib.open(nam_filepath + file)

In [4]:
# Let's see what variables are stored in the grib file
variables = []
level_type = []
for grb in grbs:
    # Create a variable for the variable name (cf convention)
    var = grb.cfName
    # Create a variable for the level type
    level = grb.typeOfLevel
    if var not in variables:
        # If there is a new variable, add it to the list!
        variables.append(var)
    else:
        # If it is not a new variable, continue looping through
        continue
    if level not in level_type:
        level_type.append(level)
    else:
        continue

### These are the different variables (cf-convention)

In [5]:
for var in variables:
    print(var)

unknown
surface_air_pressure
geopotential_height
air_temperature
relative_humidity
eastward_wind
northward_wind
surface_roughness_length
surface_upward_sensible_heat_flux
surface_upward_latent_heat_flux
land_binary_mask
surface_albedo
lagrangian_tendency_of_air_pressure


### These are the different level types

In [6]:
for lev in level_type:
    print(lev)

meanSea
surface
heightAboveGround
isobaricInhPa


In [7]:
import xarray as xr
# Read in the miroc dataset
miroc_sfc = xr.open_dataset('../preliminary_netcdf/MIROC_2d_vars.nc')
miroc_sfc

<xarray.Dataset>
Dimensions:                                  (x: 614, y: 428)
Coordinates:
    month                                    int64 ...
    lon                                      (y, x) float64 ...
    lat                                      (y, x) float64 ...
    height                                   float64 ...
Dimensions without coordinates: x, y
Data variables:
    Pressure_reduced_to_MSL_msl              (y, x) float64 ...
    Pressure_surface                         (y, x) float64 ...
    Temperature_surface                      (y, x) float64 ...
    specific_humidity                        (y, x) float64 ...
    Relative_humidity_height_above_ground    (y, x) float64 ...
    Temperature_height_above_ground          (y, x) float64 ...
    v-component_of_wind_height_above_ground  (y, x) float64 ...
    u-component_of_wind_height_above_ground  (y, x) float64 ...

#### List out the different soil levels

In [8]:
for grb in grbs.select(typeOfLevel='depthBelowLandLayer', name='Soil Temperature'):
    print(grb.level)

0
10
40
100


---
### Setup variable matching to prep for value modification

In [10]:
# Skin variables - temperature and air pressure (2-D)
skin_vars = ['air_temperature', 'surface_air_pressure']
miroc_skin_vars = ['Temperature_surface', 'Pressure_surface']
near_skin_vars = dict(zip(skin_vars, miroc_skin_vars))

In [11]:
# Variables just above Earth surface (2-D) (temperature, wind, RH)
sfc_vars = ['2 metre temperature', 'Relative humidity', 
            '10 metre U wind component', '10 metre V wind component']

miroc_vars = ['Temperature_height_above_ground', 'Relative_humidity_height_above_ground',
              'u-component_of_wind_height_above_ground', 'v-component_of_wind_height_above_ground']

miroc_dict = dict(zip(sfc_vars, miroc_vars))

In [12]:
# Different levels to loop over
levels = ['heightAboveGround', 'surface']

### This loop actually does the value extraction and modification and saves a new grib file

In [13]:
grbout = open(new_filepath+'test_surface_vars.grb','wb')
for lev in levels:
    if lev == 'heightAboveGround':
        for var in sfc_vars:
            grb = grbs.select(typeOfLevel=lev, name=var)[0]
            grb['values'] = grb.values + miroc_sfc[miroc_dict[var]].values
            msg = grb.tostring()
            grbout.write(msg)
    elif lev == 'surface':
        for var in near_sfc_vars:
            grb = grbs.select(typeOfLevel=lev, cfName=var)[0]
            grb['values'] = grb.values + miroc_sfc[near_skin_vars[var]].values
            msg = grb.tostring()
            grbout.write(msg)
grbout.close()

NameError: name 'near_sfc_vars' is not defined

## **Write to file** 

### Framework for writing to GRIB files

In [ ]:
grbout = open(new_filepath+file,'wb')
for grb in grbs:
    msg = grb.tostring()
    grbout.write(msg)
grbout.close()